In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
sns.set()

In [2]:
cust_df = pd.read_csv('takehome_users.csv', encoding='ISO-8859-1')
cust_df.head()

,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id
0,1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.398139e+09,1,0,11,10803.0
1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0
2,3,2013-03-19 23:14:52,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,1.363735e+09,0,0,94,1525.0
3,4,2013-05-21 08:09:28,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,1.369210e+09,0,0,1,5151.0
4,5,2013-01-17 10:14:20,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,1.358850e+09,0,0,193,5240.0


In [3]:
login_df = pd.read_csv('takehome_user_engagement.csv')
login_df.head()

,time_stamp,user_id,visited
0,2014-04-22 03:53:30,1,1
1,2013-11-15 03:45:04,2,1
2,2013-11-29 03:45:04,2,1
3,2013-12-09 03:45:04,2,1
4,2013-12-25 03:45:04,2,1


In [4]:
login_df.time_stamp = pd.to_datetime(login_df.time_stamp)
login_df = login_df.set_index('time_stamp')

In [5]:
new_df = login_df.groupby('user_id').rolling('7D').visited.count()

In [6]:
df = pd.DataFrame(new_df)
df = df.reset_index()
cust_df['7d_freq'] = df.groupby('user_id').max().visited

In [7]:
cust_df['target'] = np.where(cust_df['7d_freq'] > 2,1,0)
cust_df.head()

,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id,7d_freq,target
0,1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.398139e+09,1,0,11,10803.0,NaN,0
1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0,1.0,0
2,3,2013-03-19 23:14:52,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,1.363735e+09,0,0,94,1525.0,3.0,1
3,4,2013-05-21 08:09:28,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,1.369210e+09,0,0,1,5151.0,1.0,0
4,5,2013-01-17 10:14:20,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,1.358850e+09,0,0,193,5240.0,1.0,0


In [8]:
cust_df.creation_source = cust_df.creation_source.astype('category')
cust_df.org_id = np.where(cust_df.org_id > 0, 1,0)
cust_df.invited_by_user_id = np.where(cust_df.invited_by_user_id > 0, 1,0)
cust_df = cust_df.drop(['creation_time','email','name'],axis=1)
cust_df.dtypes

object_id                        int64
creation_source               category
last_session_creation_time     float64
opted_in_to_mailing_list         int64
enabled_for_marketing_drip       int64
org_id                           int32
invited_by_user_id               int32
7d_freq                        float64
target                           int32
dtype: object

In [9]:
cust_df = pd.get_dummies(cust_df)

In [10]:
cust_df.corr()

,object_id,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id,7d_freq,target,creation_source_GUEST_INVITE,creation_source_ORG_INVITE,creation_source_PERSONAL_PROJECTS,creation_source_SIGNUP,creation_source_SIGNUP_GOOGLE_AUTH
object_id,1.000000,0.015238,-0.032370,-0.022040,-0.013565,-0.009149,-0.000223,0.005863,-0.006909,-0.003988,-0.003357,0.002692,0.015089
last_session_creation_time,0.015238,1.000000,0.006590,0.008035,0.048954,0.004916,0.003245,0.001123,0.015054,-0.006940,0.001361,-0.006164,-0.000824
opted_in_to_mailing_list,-0.032370,0.006590,1.000000,0.483529,0.003101,0.006934,-0.009024,-0.003227,0.000667,0.006694,-0.007432,0.002181,-0.004555
enabled_for_marketing_drip,-0.022040,0.008035,0.483529,1.000000,0.008194,0.002216,0.022893,0.012902,-0.002438,0.004270,-0.007518,0.004528,0.000127
org_id,-0.013565,0.048954,0.003101,0.008194,1.000000,0.012032,0.015517,0.008509,0.003369,0.009839,-0.016167,0.006122,-0.006780
invited_by_user_id,-0.009149,0.004916,0.006934,0.002216,0.012032,1.000000,-0.000502,0.002127,0.437386,0.691238,-0.495336,-0.491916,-0.387255
7d_freq,-0.000223,0.003245,-0.009024,0.022893,0.015517,-0.000502,1.000000,0.920458,0.012884,-0.010899,-0.015036,0.004420,0.013436
target,0.005863,0.001123,-0.003227,0.012902,0.008509,0.002127,0.920458,1.000000,0.012901,-0.008149,-0.015971,0.004128,0.010814
creation_source_GUEST_INVITE,-0.006909,0.015054,0.000667,-0.002438,0.003369,0.437386,0.012884,0.012901,1.000000,-0.347502,-0.216653,-0.215157,-0.169380
creation_source_ORG_INVITE,-0.003988,-0.006940,0.006694,0.004270,0.009839,0.691238,-0.010899,-0.008149,-0.347502,1.000000,-0.342395,-0.340031,-0.267686


In [12]:
cust_df['7d_freq'] = cust_df['7d_freq'].fillna(0)
cust_df = cust_df.dropna()

In [13]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
X = cust_df.drop(['7d_freq','target'],axis=1)
y = cust_df.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [14]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_X = scaler.fit_transform(X_train)

In [15]:
clf = LogisticRegression(random_state=42).fit(scaled_X,y_train)
X_test_scaled = scaler.transform(X_test)
clf.score(X_test_scaled,y_test)

0.8696883852691218

In [16]:
features = list(cust_df.columns)
features.remove('7d_freq')
features.remove('target')
coefs_df = pd.DataFrame(clf.coef_, columns = features).transpose().rename(columns={0:'Features'})
coefs_df

,Features
object_id,0.004382
last_session_creation_time,-0.014812
opted_in_to_mailing_list,0.014170
enabled_for_marketing_drip,0.059311
org_id,0.071920
invited_by_user_id,-0.002685
creation_source_GUEST_INVITE,0.061644
creation_source_ORG_INVITE,-0.052321
creation_source_PERSONAL_PROJECTS,-0.070007
creation_source_SIGNUP,0.023696
